In [1]:
import os
import sys
from pathlib import Path

cwd = Path(os.getcwd())
if cwd.name != "backend":
    repo = cwd.parent.parent
    sys.path.append(str(cwd))
    sys.path.append(str(repo))
    
    os.chdir(repo / "src" / "backend")

from data.evaluation.src.jupyter_utils.preamble import *
from importlib import reload


OCEL_KEYS = [
    "containerLogistics",
    "orderManagementWithDistances",
    "p2p",
    "hinge",
]

In [2]:
import gc
import src.eva_util as eva  # type: ignore
import src.eva_stats as stats  # type: ignore

reload(ag)
reload(ar)
reload(allocation)
reload(export)
reload(stats)
reload(eva)

RuleName = Literal["AllTargets", "ParticipatingTargets", "ClosestTargets"]

def run_ocel_allocation(
    ocel: OCELWrapper,
    ocel_key: str,
    hu_otypes: set[str],
    resource_otypes: set[str],
    skip_params: list[tuple[Any]] = [],
    filter_rules: list[RuleName] | None = None,
    filter_graph_mode: list[ag.GraphMode] | None = None,
    filter_remove_otype_loops: list[bool] | None = None,
):

    tots = sorted(hu_otypes) + sorted(resource_otypes)
    # tots = tots[:1]

    if any(f is not None for f in [filter_rules, filter_graph_mode, filter_remove_otype_loops]):
        raise NotImplementedError

    rules: dict[RuleName, Callable[..., ar.AllocationRule]] = {
        "AllTargets": lambda alloc, _1, _2: ar.AllTargetsAllocation(alloc),
        "ParticipatingTargets": lambda alloc, _1, _2: ar.ParticipatingTargetsAllocation(alloc),
        "ClosestTargets": lambda alloc, graph_mode, remove_otype_loops: ar.ClosestTargetsAllocation(
            alloc,
            graph_mode=graph_mode,
            remove_otype_loops=remove_otype_loops,
            max_distance=10,
            algorithm="own_bfs",
            progress=True,
        ),
    }
    
    for tot in tots:
        for rule, rule_factory in rules.items():
            if rule == "ClosestTargets":
                _remove_otype_loops = [True, False]
                # _remove_otype_loops = [False]
                # _graph_modes = [ag.GraphMode.HU_HU]
                _graph_modes = [ag.GraphMode.HU_HU, ag.GraphMode.OBJ_OBJ]
            else:
                _remove_otype_loops = [None]
                _graph_modes = [None]
            for graph_mode in _graph_modes:
                for remove_otype_loops in _remove_otype_loops:
                    graph_mode_rep = str(graph_mode) if graph_mode is not None else None
                    if (ocel_key, tot, rule, graph_mode_rep, remove_otype_loops) in skip_params:
                        print(
                            f"Skipping ({ocel_key}, {tot}, {rule}, {graph_mode_rep}, {remove_otype_loops}) ..."
                        )
                        continue
                    params = {
                        k: v
                        for k, v in dict(
                            graph_mode=graph_mode, remove_otype_loops=remove_otype_loops
                        ).items()
                        if v != None
                    }
                    param_info = (
                        ("(" + ", ".join([f"{k}={v}" for k, v in params.items()]) + ")")
                        if params
                        else ""
                    )
                    print(f"Allocating to target type '{tot}' using rule '{rule}' {param_info} ...")

                    alloc = allocation.Allocator.dummy(
                        ocel,
                        {tot},
                        hu_otypes,
                        resource_otypes,
                        events=1,
                        rule=lambda alloc: rule_factory(alloc, graph_mode, remove_otype_loops),
                        silent=True,
                    )

                    alloc.process()

                    # compute statistics
                    # TODO add more!
                    # Need all data that are needed to generate figures.
                    # Don't try generating figures for all params at once!

                    name_suffix = f"{tot}_" + eva.get_param_str(
                        rule=rule,
                        graph_mode=graph_mode,
                        remove_otype_loops=remove_otype_loops,
                        mode="file",
                    )

                    graph_stats = {}
                    if isinstance(alloc.rule, ar.ClosestTargetsAllocation):
                        graph_stats.update(
                            dict(
                                og_nodes=alloc.rule.OG.order(),
                                og_edges=alloc.rule.OG.size(),
                                og_components=len(list(nx.connected_components(alloc.rule.OG))),
                            )
                        )
                    stats_meta = dict(
                        name_suffix=name_suffix,
                        ocel_key=ocel_key,
                        meta=dict(
                            rule=rule,
                            target_otypes=util.set_str(alloc.target_otypes),
                            graph_mode=graph_mode,
                            remove_otype_loops=remove_otype_loops,
                            hu_otypes=util.set_str(alloc.hu_otypes),
                            resource_otypes=util.set_str(alloc.resource_otypes),
                            init_time=alloc._init_timer.time,
                            process_time=alloc._process_timer.time,
                            **graph_stats,
                        ),
                    )

                    assert alloc.event_stats is not None
                    # display(alloc.event_stats)
                    if isinstance(alloc.rule, ar.ClosestTargetsAllocation):
                        display(alloc.rule.object_stats)

                        targets_per_act_and_dist = stats.targets_per_act_and_dist(
                            alloc, alloc.event_stats
                        )

                        # display(stats.format(targets_per_act_and_dist, stats.targets_per_act_and_dist))
                        export.save_ocel_stats(
                            targets_per_act_and_dist,
                            name="targets_per_act_and_dist",
                            **stats_meta,  # type: ignore
                        )

                    # Export target emissions
                    eva.export_alloc_results(
                        alloc,
                        ocel_key,
                        tot,
                        rule,
                        graph_mode,
                        remove_otype_loops,
                        init_time=alloc._init_timer.time,
                        process_time=alloc._process_timer.time,
                        report=False,
                        meta=stats_meta["meta"],  # type: ignore
                    )

                    if isinstance(alloc.rule, ar.ClosestTargetsAllocation):
                        del alloc.rule.OG
                    del alloc
                    gc.collect()

In [ ]:
skip_after_timestamp = datetime.strptime("20240823-140000", "%Y%m%d-%H%M%S")
"""Skip runs with the same parameters that have already been executed after this minimum timestamp.

Use a future date to force all new computations."""


for OCEL_KEY in OCEL_KEYS:
    if OCEL_KEY == "containerLogistics":
        from data.evaluation.src.ocel_config.container_logistics import *
    elif OCEL_KEY == "orderManagementWithDistances":
        from data.evaluation.src.ocel_config.order_management_with_distances import *
    elif OCEL_KEY == "p2p":
        from data.evaluation.src.ocel_config.p2p import *
    elif OCEL_KEY == "hinge":
        from data.evaluation.src.ocel_config.hinge import *
    else:
        raise ValueError
    
    print("OCEL:", OCEL_KEY, "\n")

    otypes = set(ocel.otypes)
    assert hu_otypes.isdisjoint(resource_otypes), "HU types and Resource types are not disjoint."
    assert hu_otypes.union(resource_otypes) == otypes, "HU types and Resources do not form a complete partition."

    print(f"Target object type(s): {util.set_str(target_otypes)}")
    print(
        f"HUs: {util.set_str(sorted(hu_otypes, key=lambda ot: (0, ot) if ot in target_otypes else (1, ot)))}"
    )
    print(f"Resources: {util.set_str(resource_otypes)}")

    # Determine what parameters have already been tested
    try:
        _, stats_meta, (tmin, tmax) = export.load_ocel_stats(
            name="alloc_target_emissions",
            min_timestamp=skip_after_timestamp,
            ocel_key=OCEL_KEY,
            target_otypes=None,
            rule=None,
            graph_mode=None,
            remove_otype_loops=None,
            load=False,
        )  # type: ignore

        stats_meta = stats_meta.fillna(np.nan).replace([np.nan], [None])
        params = ["ocel", "target_otypes", "rule", "graph_mode", "remove_otype_loops"]
        skip_params = list(
            map(
                tuple,
                stats_meta[params].to_dict("tight")["data"],  # type: ignore
            )
        )
        print(f"Min time: {tmin}")
        print(f"Max time: {tmax}")
        print(f"Number of runs to be skipped: {len(stats_meta)}")
        print(f"Number of runs per target otype:")
        display(stats_meta.groupby(["ocel", "target_otypes"]).size())
        
    except ValueError:
        print("No exports found matching the filter.")
        skip_params = []

    run_ocel_allocation(
        ocel=ocel,
        ocel_key=OCEL_KEY,
        hu_otypes=hu_otypes,
        resource_otypes=resource_otypes,
        skip_params=skip_params,
    )